Муравьиные алгоритмы


In [67]:
# solution = [[[69, 40, 50, 33, 31, 8, 55, 60, 69], [52, 7, 2, 82, 74, 60, 72, 26, 52]], 
#             [[69, 50, 40, 60, 55, 8, 31, 33, 69], [52, 2, 7, 26, 72, 60, 74, 82, 52]], 
#             [[69, 33, 8, 31, 55, 60, 50, 40, 69], [52, 82, 60, 74, 72, 26, 2, 7, 52]], 
#             [[69, 33, 31, 8, 55, 50, 40, 60, 69], [52, 82, 74, 60, 72, 2, 7, 26, 52]], 
#             [[69, 60, 50, 40, 55, 33, 8, 31, 69], [52, 26, 2, 7, 72, 82, 60, 74, 52]], 
#             [[69, 55, 8, 31, 33, 40, 50, 60, 69], [52, 72, 60, 74, 82, 7, 2, 26, 52]], 
#             [[69, 55, 33, 31, 8, 60, 50, 40, 69], [52, 72, 82, 74, 60, 26, 2, 7, 52]], 
#             [[69, 55, 31, 33, 8, 40, 50, 60, 69], [52, 72, 74, 82, 60, 7, 2, 26, 52]], 
#             [[69, 55, 33, 31, 8, 40, 50, 60, 69], [52, 72, 82, 74, 60, 7, 2, 26, 52]]]

In [57]:
d_cities = {
    "A": [69, 52],
    "B": [50, 2],
    "C": [8, 60],
    "D": [31, 74],
    "E": [60, 26],
    "F": [40, 7],
    "G": [55, 72],
    "H": [33, 82]
}

In [58]:
d_cities

{'A': [69, 52],
 'B': [50, 2],
 'C': [8, 60],
 'D': [31, 74],
 'E': [60, 26],
 'F': [40, 7],
 'G': [55, 72],
 'H': [33, 82]}

Сверху были тестовые данные. Теперь мы уже умеем работать с классом TSP

In [59]:
from algorithms.ant import AntColony
from tsp import TSP
from typing import List

In [60]:
cities: List[TSP.City] = []
for city, coords in d_cities.items():
    id, x, y = city, coords[0], coords[1]
    cities.append(TSP.City(id, int(x), int(y)))

tsp = TSP(cities)
initial_state = TSP.State(1 << 0, 0)

as_settings = AntColony.Settings()
as_colony = AntColony(AntColony.Variation.ANT_SYSTEM, as_settings)

In [63]:
%%time
path, dist = as_colony.solve(initial_state, tsp.successors, tsp.goal, tsp.add_generation)
print("Rank-based Ant System: ", [p.current_node for p in path], dist)

Rank-based Ant System:  [0, 6, 7, 3, 2, 5, 1, 4, 0] 210.35643810611498
CPU times: user 12.5 s, sys: 0 ns, total: 12.5 s
Wall time: 12.5 s


In [64]:
import pandas as pd

df = pd.DataFrame(tsp.cities_to_dict().items(), columns=['name', 'coords'])
df[['x', 'y']] = pd.DataFrame(df.coords.tolist(), index=df.index)
df.drop(columns=['coords'], inplace=True)
df

,name,x,y
0,A,69,52
1,B,50,2
2,C,8,60
3,D,31,74
4,E,60,26
5,F,40,7
6,G,55,72
7,H,33,82


In [66]:
import plotly.graph_objects as go

points = go.Scatter(x=df['x'], y=df['y'], mode='markers',
                    marker=dict(color='#5D69B1', size=10), name='Город',
                    text = ["Название: {}".format(name) for name in df['name'] ])

paths_history = tsp.get_solution_in_generations()
generations = len(paths_history)

path = go.Scatter(x=paths_history[generations - 1][0], y=paths_history[generations - 1][1], 
                  mode='lines', name="Маршрут")

button = {
        "type": "buttons",
        "buttons": [
            {
                "label": "Запустить",
                "method": "animate",
                "args": [None, {"frame": {"duration": 180}}],
            }
        ],
    }

frames = []
for generation in range(generations):
    path = go.Scatter(x=paths_history[generation][0], y=paths_history[generation][1], 
                      mode='lines', name="Маршрут")
    layout = go.Layout(updatemenus=[button], 
                       title_text=f"Маршрут, найденный муравьиным алгоритмом за {generation + 1} итерации")
    frame = go.Frame(data=[points, path], layout=layout)
    frames.append(frame)


fig = go.Figure(data=[points, path], frames=frames, layout=layout)
fig.show()